# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("../WeatherPy/Output_Data/weather.csv")
weather

,City,Lat,Lng,Max_Temp,Humidity,Speed,Date,Cloudiness,Country
0,umm lajj,25.0213,37.2685,82.06,55,4.36,04/24/2021,0,SA
1,mount gambier,-37.8333,140.7667,50.00,100,8.05,04/24/2021,75,AU
2,vaini,-21.2000,-175.2000,69.80,100,6.22,04/24/2021,20,TO
3,kirakira,-10.4544,161.9205,78.82,82,1.68,04/24/2021,99,SB
4,tasiilaq,65.6145,-37.6368,37.40,75,4.61,04/24/2021,75,GL
...,...,...,...,...,...,...,...,...,...
571,blagoyevo,63.3667,47.9167,46.96,64,7.11,04/24/2021,34,RU
572,camana,-16.6228,-72.7111,71.02,71,9.80,04/24/2021,86,PE
573,krutinka,56.0055,71.5113,41.77,50,9.35,04/24/2021,100,RU
574,maniwaki,46.3834,-75.9660,66.20,26,10.36,04/24/2021,44,CA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [26]:
coordinates = weather[["Lat", "Lng"]]
humidity = weather["Humidity"]


fig = gmaps.figure()
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, 
                                 dissipating = False, max_intensity = 10, point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_ideal = weather[(weather["Max_Temp"] >70) & 
                        (weather["Max_Temp"] <80) & 
                        (weather["Speed"] <10) &
                        (weather["Cloudiness"] ==0)]
weather_ideal

,City,Lat,Lng,Max_Temp,Humidity,Speed,Date,Cloudiness,Country
30,san juan,-31.5375,-68.5364,73.11,29,7.47,04/24/2021,0,AR
132,beira,-19.8436,34.8389,73.40,83,8.81,04/24/2021,0,MZ
364,jaspur,29.2833,78.8167,75.54,27,5.66,04/24/2021,0,IN
471,maxixe,-23.8597,35.3472,70.56,81,6.82,04/24/2021,0,MZ


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_ideal.reset_index()
del hotel_df["index"]
hotel_df

hotel_df["Hotel Name"]=""
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Speed,Date,Cloudiness,Country,Hotel Name
0,san juan,-31.5375,-68.5364,73.11,29,7.47,04/24/2021,0,AR,
1,beira,-19.8436,34.8389,73.40,83,8.81,04/24/2021,0,MZ,
2,jaspur,29.2833,78.8167,75.54,27,5.66,04/24/2021,0,IN,
3,maxixe,-23.8597,35.3472,70.56,81,6.82,04/24/2021,0,MZ,


In [7]:
print(type(hotel_df))

<class 'pandas.core.frame.DataFrame'>


In [8]:
# target_coordinates = "-31.5375, -68.5364"
target_radius = 5000
target_type = "lodging"


params = {"radius": target_radius,
          "type": target_type,
          "key": g_key}


# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# response = requests.get(base_url, params=params)
# near_hotel = response.json()
# # print(json.dumps(near_hotel, indent = 4, sort_keys=True))

# print(near_hotel["results"][0]["name"])



for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] =f"{lat} {lng}"
      
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    near_hotel = response.json()
#     print(json.dumps(near_hotel, indent = 4, sort_keys = True))
    
    results = near_hotel["results"]
    print(near_hotel["results"][0]["name"])
    try:
        print(near_hotel["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except: 
        print("no result")
        pass

hotel_df

Hotel Provincial
Hotel Provincial
Beira Terrace Hotel
Beira Terrace Hotel
Chaudhary Restaurant And Hotel
Chaudhary Restaurant And Hotel
Oceano
Oceano


,City,Lat,Lng,Max_Temp,Humidity,Speed,Date,Cloudiness,Country,Hotel Name
0,san juan,-31.5375,-68.5364,73.11,29,7.47,04/24/2021,0,AR,Hotel Provincial
1,beira,-19.8436,34.8389,73.40,83,8.81,04/24/2021,0,MZ,Beira Terrace Hotel
2,jaspur,29.2833,78.8167,75.54,27,5.66,04/24/2021,0,IN,Chaudhary Restaurant And Hotel
3,maxixe,-23.8597,35.3472,70.56,81,6.82,04/24/2021,0,MZ,Oceano


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
# Display figure
figure_layout = {
    'width': '450px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, 
                                 dissipating = False, max_intensity = 100, point_radius = 1)
fig.add_layer(heat_layer)

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…